# Sample Hangul RNN

In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import string
import argparse
import time
import os
import hgtk

from six.moves import cPickle
from TextLoader import *

print ("Packages Imported")

Packages Imported


# Load dataset using TextLoader

In [2]:
data_dir    = "data/nine_dreams"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "data/nine_dreams"   
#  from "data/nine_dreams/input.txt" 
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "\n" )
print ("data_loader.vocab looks like \n%s " %
       (data_loader.vocab))
print ( "\n" )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )
print ( "\n" )
print ("data_loader.chars looks like \n%s " % (data_loader.chars,))

loading preprocessed files
type of 'data_loader' is <class 'dict'>, length is 76


data_loader.vocab looks like 
{'ㄲ': 29, 'ㄺ': 48, ')': 46, 'ㅄ': 43, '1': 58, 'ㅡ': 8, 'ㅛ': 38, 'ㄸ': 32, 'ㅓ': 11, 'ㅌ': 30, 'ㅚ': 33, '2': 62, '"': 28, '?': 41, ' ': 1, ',': 27, 'ㅑ': 34, 'ㅏ': 3, 'ㅅ': 10, 'ㅋ': 42, 'ㅂ': 17, '(': 45, 'ㄾ': 70, ':': 57, 'ㄼ': 54, '0': 73, 'ㄴ': 4, 'ᴥ': 0, 'ㄳ': 60, '_': 69, 'ㅃ': 47, 'ㅁ': 12, 'ㅞ': 61, 'ㅜ': 16, 'ㄹ': 5, 'ㄷ': 13, 'ㄵ': 50, 'ㅘ': 26, '7': 71, '\x1a': 72, 'ㅀ': 51, 'ㅒ': 74, 'ㅖ': 39, '-': 65, 'ㄱ': 7, '6': 59, '>': 75, 'ㅣ': 6, 'ㅐ': 21, 'ㅉ': 40, '.': 24, 'ㅇ': 2, 'ㅔ': 20, 'ㅈ': 15, '9': 64, '4': 67, 'ㅠ': 37, 'ㅎ': 14, '5': 63, 'ㅕ': 18, 'ㅝ': 36, "'": 49, '\n': 19, 'ㄶ': 44, 'ㅗ': 9, 'ㅢ': 25, 'ㅍ': 31, '3': 66, 'ㄻ': 55, 'ㅙ': 53, 'ㄿ': 56, 'ㅊ': 23, '!': 52, '8': 68, 'ㅆ': 22, 'ㅟ': 35} 


type of 'data_loader.chars' is <class 'tuple'>, length is 76


data_loader.chars looks like 
('ᴥ', ' ', 'ㅇ', 'ㅏ', 'ㄴ', 'ㄹ', 'ㅣ', 'ㄱ', 'ㅡ', 'ㅗ', 'ㅅ', 'ㅓ', 'ㅁ', 'ㄷ', 'ㅎ', 'ㅈ', 'ㅜ', 'ㅂ', 'ㅕ', '\n', 'ㅔ', 'ㅐ', 

# Define Network

In [3]:
rnn_size   = 512
num_layers = 3
grad_clip  = 5.

_batch_size = 1
_seq_length = 1

vocab_size = data_loader.vocab_size

with tf.device("/cpu:0"):
    # Select RNN Cell
    unitcell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([unitcell] * num_layers)
    # Set paths to the graph 
    input_data = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    targets    = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    initial_state = cell.zero_state(_batch_size, tf.float32)

    # Set Network
    with tf.variable_scope('rnnlm'):
        softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
        softmax_b = tf.get_variable("softmax_b", [vocab_size])
        with tf.device("/cpu:0"):
            embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
            inputs = tf.split(tf.nn.embedding_lookup(embedding, input_data), _seq_length, 1)
            inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
            
    # Loop function for seq2seq
    def loop(prev, _):
        prev = tf.nn.xw_plus_b(prev, softmax_w, softmax_b)
        prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
        return tf.nn.embedding_lookup(embedding, prev_symbol)
    # Output of RNN 
    outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state
                                , cell, loop_function=None, scope='rnnlm')
    output = tf.reshape(tf.concat(outputs, 1), [-1, rnn_size])
    logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
    # Next word probability 
    probs = tf.nn.softmax(logits)
    # Define LOSS
    loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], # Input
        [tf.reshape(targets, [-1])], # Target
        [tf.ones([_batch_size * _seq_length])], # Weight 
        vocab_size)
    # Define Optimizer
    cost = tf.reduce_sum(loss) / _batch_size / _seq_length
    final_state = last_state
    lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    _optm = tf.train.AdamOptimizer(lr)
    optm = _optm.apply_gradients(zip(grads, tvars))

print ("Network Ready")

Network Ready


In [4]:
# Sample ! 
def sample( sess, chars, vocab, __probs, num=200, prime=u'ㅇㅗᴥㄴㅡㄹᴥ '):
    state = sess.run(cell.zero_state(1, tf.float32))
    _probs = __probs
    prime = list(prime)
    for char in prime[:-1]:
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [state] = sess.run([final_state], feed)

    def weighted_pick(weights):
        weights = weights / np.sum(weights) 
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime
    char = prime[-1]
    for n in range(num):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [_probsval, state] = sess.run([_probs, final_state], feed)
        p = _probsval[0]
        sample = int(np.random.choice(len(p), p=p))
        # sample = weighted_pick(p)
        # sample = np.argmax(p)
        pred = chars[sample]
        ret += pred
        char = pred
    return ret
print ("sampling function done.")

sampling function done.


# Sample

In [5]:
save_dir = 'data/nine_dreams'
prime = hgtk.text.decompose(u"누구 ")

print ("Prime Text : %s => %s" % (hgtk.text.compose(prime), "".join(prime)))
n = 2000

sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state(save_dir)

# load_name = u'data/nine_dreams/model.ckpt-0'
# load_name = u'data/nine_dreams/model.ckpt-99000'
load_name = tf.train.latest_checkpoint(save_dir)

print (load_name)

if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, load_name)
    sampled_text = sample(sess, chars, vocab, probs, n, prime)
    #print ("")
    print (u"SAMPLED TEXT = %s" % "".join(sampled_text))
    print ("")
    print ("-- RESULT --")
    print (hgtk.text.compose("".join(sampled_text)))

누
구
 
Prime Text : 누구  => ㄴㅜᴥㄱㅜᴥ 
data/nine_dreams/model.ckpt-0
SAMPLED TEXT = ㄴㅜᴥㄱㅜᴥ :ㅢ16ㄵ?ㄲㅓ
ㄵㅖ1ㄳㅀㄾㅞ7ㅄㅆㅞ.ㅡㅄ ㅉㄹㄴㅉㅙ:ㄵ)ㅛㅄㅊ7ㅏㄼ.ㄸㅚ
ㅕ?0ㅉㅓ

ㅡㅇ?7.
.7ㄼㅛㅚ7ㅎㅏㄹ.
ㄳㄷㅞㅞㅆ.ㅣㄶㅒㄵ:ㅛㄳㅊ
ㅇㅊㄻㄵㅁㄻㄾㅖ
ㅓㄴㅖㄴㅛㄴㅟㄶㅃㄶㅘㅊㄹ.ㅊㄲ:ㅖ.ㅌㄻㄸㄸㄴ)6!ㄿㄾㅛㅛㅘㄶㅊㅆㅘㅠㅖㅊㅉㅄㄼㅚㅏ.ㅞ?ㅉㅡㅠㅛㅏㅊㅓ?ㄸ.ㄸ107.ㅛㅞ5ㅏㄹㅆㄲㅓㅛㅃㄴ)ㅖ(ㅘㄿㄲㅏㄴㄼㅇ15ㄵㅣㅞ1ㄹㄹㅆ1ㄴ1ㅊㅆ
ㅒㅐㅞㅓㄴ
ㅞㅖ81ㅓㅛ?ㅀ3ㅢ0ㅏㅊ'ㅏㅚㄵ0ㅚㄲㅘㅞㅆㄿㅊ6ㅍㅛ7ㅘㅡ8ㅠ7.ㅐㅖㅖ7ㅞㅎㄴ'_ㅖㄴㅆㅛㅛ.8:ㄶㅊ'ㅏㄵ3.ㄲㅑㅖ:ㅉ:1ㅊㄴ73ㅄㅀㄱㄷㄶㅍㅡㅠㅍㄵ00ㅚㅉ.ㅉㅊㄼㅊ

ㅖ!7ㅛ1ㄾㄹㅔ':ㅍㄲㅐ_ㅊㄶㅍ6ㅍ7ㅃㅡ6ㅃㄴㅡㅒ(5ㅁㅇ7ㅛㄾ
ㅃㄵㄿ0ㄱㄳㄹ
"_
.ㅊㅆ6ㄾ5ㅉ.ㅄㄴㅛㄳ?,ㅆ'ㅊㄺ
ㄼㅞㄵㅓㄹㄲ.,ㅛㅄ:ㄴㅆ.ㅖㅠㅍ7ㅃㄹㅋㅖㅚㅞㅆ.ㅛ1?ㄳㅒㅓㄳㅞ.ㅠㅣㅆ?ㅆㄹㄲㅌㅃㄲㄻㅇㅌ5ᴥㅉㅛ?ㅊ,ㅖㅁ>ㄸ
!ㅒㅒㅛㅆㄴㅓ?ㅒㅎ.ㅚ
!ㄲㅞㅣㄵㄵㅎ71
))ㅉㄳ.ㅈㄹ)ㅔㄹ1ㄴㄷ4ㄻㄲㅄㅄㅅㅣㄼㅍㅊ.:5-ㄶㄹㅆ.ㅆㅠㄲㅎ6)!
ㄳㄴㅖㄲㅝ1'_ㅛㅝㅉㅇㅏㅆ.ㅛㅖㅃ.ㅒㅈㅃㄴㄹㅚㄾㅇㅖㄶㅊㄾㅘㅞㅛ.ㅆㄹ!
ㅏ!ㅁㅡㄴ-ㅖㄲ1ㅆㅍㅖ1.ㅎ7 ㅡㄲ:ㅖ7ㅖㄺㅚ)ㅇㅛㄶㄲ3'.ㅉ7ㅖ, ㅖㅍㅐ.ㅆ
ㅙㅖㅎㅆㅞ
ㅆㄳ-:ㅡ1ㅣㄴㄺㄶ-ㄲ. ㄻㅚ.)ㅏㅆ.
ㅏ:ㅃㄳ:ㅛㅛ
ᴥ7-!.ㄴ15ㅓ6,ㄶㄸㅄㅀ.'3ㄹㄾㅡㅛㅆㅅㅏㄴㅎㅇㅊ:ㅆ?ㄵㄵㅎ7.ㅡ1'ㅓㄹㅚㅣㄶㅄ,77ㄹㅞㄱㅓ
-.ㅍㄴ-ㄸ'3:ㅚ
ㅂ9-ㅒㅛㅚㅆ!
33ㄴ:ㅛㅡ
ㅞ3ㅘ7ㅉㄵㅁㅖ
ㅓ
ㅖㅎㅘ0ㄶ:ㄹㅡㅇㄸㅓㄴㄱㅡ!ㅃㅔㅎ>ㅆㅓㅇ,: ㅡㄹ.ㅑㄴㅃㄹㄺㅛㄺㅙ-ㅆ.ㅛ7ㅇㄴ1ㄲㄶ0ㅋ8
:,ㅆ ㅚ
ㅖㅐㄳ-ㅡ?ㅕㅂㄹㄼㅏㅉㄲㄹㅍㅞㄶㅡ

ㅛ!ㅉㄵㅘㅓ1ㅏㅖ:ㅏㅡㄾㅚㄻㄸㅡㅘㅛㅆㅍㄳㅉㄾ:ㅉ4ㅢㅃ!ㄲ
ㅘㅛ
ㅚㅊㅍㅏㄴ.ㅖ7ㄻㅕ.ㅉㅡㄴ
ㅛ1"?6ㅖㄴ
:?'ㅡㅓㅒㄴㅊㅆ.ㄸㅚ
.ㅛㄳㅆㅆㅉㅞㅕ75_ㄸ.ㄶ'ㅘㅔㄻㅕㅊㄿㄹ,ㅆ'ㅃㅏㄲㅗㅍ1
ㄶㅊ-ㅇㄵㅛ5ㄷㄹㅏㅎㅈㅛㅛ08ㅉㄹㅍ0ㄷㅖ5
ㄺ'ㅡ?ㅣ!ㅘㅎㄴㄵ,


ㅡ1